In [1]:
%%HTML
<button onclick="$('.input, .prompt, .output_stderr, .output_error, .output_result').toggle();">隐藏/显示代码</button>

In [2]:
# 标签系统
## 目标函数,1日收益率 3日收益率 5日收益率，10日收益率
## 函数功能 目标排序大于阈值的股票，标签按收益率累加权重,绘制标签云图

In [5]:
!pip install pyecharts==1.8.1 --user


You should consider upgrading via the 'C:\Users\louis.xu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import datetime as dt
out=widgets.Output()
import warnings
warnings.filterwarnings('ignore')
import pyecharts.globals as g
g.WarningType.ShowWarning=False

In [19]:
sys.path.append("/Users/louis/Pycharm/rqalpha/")

from rqalpha.data.bar_dict_price_board import BarDictPriceBoard
from rqalpha.data.base_data_source import BaseDataSource
from rqalpha.data.data_proxy import DataProxy
from rqalpha.environment import Environment

env = Environment({})
env.trading_dt = dt.datetime.now()
data_proxy = DataProxy(BaseDataSource("C:/Users/louis.xu/.rqalpha/bundle", {}), BarDictPriceBoard)

In [20]:
data_proxy.all_instruments(["CS"])


code=Materials, sector_code_name=原材料, industry_code=C26, industry_name=化学原料及化学制品制造业, concept_names=None, board_type=MainBoard, status=Active, special_type=Normal, contract_multiplier=1, margin_rate=1, underlying_order_book_id=None, underlying_symbol=None, maturity_date=None, settlement_method=None, listing=True, listed=True, de_listed=False, account_type=STOCK, trading_hours=[TimeRange(start=datetime.time(9, 31), end=datetime.time(11, 30)), TimeRange(start=datetime.time(13, 1), end=datetime.time(15, 0))], trade_at_night=False),
 Instrument(order_book_id=002456.XSHE, symbol=欧菲光, round_lot=100, listed_date=2010-08-03 00:00:00, de_listed_date=2999-12-31 00:00:00, type=CS, exchange=XSHE, market_tplus=1, sector_code=InformationTechnology, sector_code_name=信息技术, industry_code=C39, industry_name=计算机、通信和其他电子设备制造业, concept_names=None, board_type=MainBoard, status=Active, special_type=Normal, contract_multiplier=1, margin_rate=1, underlying_order_book_id=None, underlying_symbol=None, maturity_da

In [4]:
import pyecharts.options as opts
from pyecharts.charts import WordCloud


class Tags:
    def __init__(self,date):
        self.df_pivot = get_all_securities(date=date)
        self.date = date
        self.code_tags={code:set() for code in self.df_pivot.index}
        self.tag_codes={}
        self.calc_return()

    def add_series(self,name,series):
        """
        基础数据添加1列
        :param name:列名
        :param series: 数据
        :return:
        """
        self.df_pivot[name]=series
        

    def add_tag(self,name,codes):
        for code in codes:
            if code in self.code_tags:
                self.code_tags[code].add(name)
        self.tag_codes[name]=set(codes)

    def add_tag_by_expr(self,name,expr):
        """
        添加标签
        :param name:
        :param expr:
        :return:
        """
        codes=list(self.df_pivot.query(expr).index)
        for code in codes:
            self.code_tags[code].add(name)
        self.tag_codes[name]=set(codes)

    def calc_return(self):
        """
        计算指定日期的1,3,5,10日收益率
        :return:
        """
        panel=get_price(list(self.df_pivot.index),end_date=self.date,count=11,fields=['close','volume'])
        close=panel['close']
        volume=panel['volume']
        df_vol_5=volume.rolling(5).mean()
        self.add_series('return1',(close.iloc[-1]-close.iloc[-2])/close.iloc[-2])
        self.add_series('return3',(close.iloc[-1]-close.iloc[-4])/close.iloc[-4])
        self.add_series('return5',(close.iloc[-1]-close.iloc[-6])/close.iloc[-6])
        self.add_series('return10',(close.iloc[-1]-close.iloc[-11])/close.iloc[-11])
        self.add_series('vol1', (df_vol_5.iloc[-1] - df_vol_5.iloc[-2]) / df_vol_5.iloc[-2]*100)
        self.add_series('vol3', (df_vol_5.iloc[-1] - df_vol_5.iloc[-4]) / df_vol_5.iloc[-4]*100)
        self.add_series('vol5', (df_vol_5.iloc[-1] - df_vol_5.iloc[-6]) / df_vol_5.iloc[-6]*100)
        self.add_series('vol10', (df_vol_5.iloc[-1] - df_vol_5.iloc[-11]) / df_vol_5.iloc[-11]*100)

    def stat(self,expr="return3>0.05",target_col='return3'):
        """
        统计各tag得分
        :param expr: 收益率过滤函数
        :param target_col: 权重列
        :return:
        """
        df_select=self.df_pivot.query(expr)
        all_tags=set()
        for codeset in self.code_tags.values():
            all_tags=all_tags|codeset
        tag_weight={tag:0 for tag in all_tags}
        for index,row in df_select.iterrows():
            for tag in self.code_tags[index]:
                tag_weight[tag]=tag_weight[tag]+row[target_col]
        for tag in tag_weight.keys():
            tag_weight[tag]=tag_weight[tag]/max(len(self.tag_codes[tag]),5)
        return list(tag_weight.items())

    def get_stock_from_tag(self,tag):
        return self.df_pivot[self.df_pivot.index.isin(list(self.tag_codes[tag]))]
        
    def wordcloud(self,filter_expr="return3>0.05",sort_col='return3'):
        data=self.stat(filter_expr,sort_col)
        return WordCloud(init_opts=opts.InitOpts(width='100%', height='600px'))\
        .add(series_name="热点概念", data_pair=data, word_size_range=[8, 100],shape='circle',width='95%',height='550px',rotate_step=5)\
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="热点概念", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
            ),
            tooltip_opts=opts.TooltipOpts(is_show=True),

        ).render_notebook()

In [ ]:
from jqdata import *
import pandas as pd
def all_concepts():
    """聚源数据所有概念"""
    dfs=[]
    min_id=9953668143482
    while len(dfs)<30 and min_id>0:
        q = query(
                jy.LC_COConcept
            ).filter(jy.LC_COConcept.IndiState==1,jy.LC_COConcept.ID<min_id).order_by(jy.LC_COConcept.ID.desc())
        df = jy.run_query(q)
        min_id=df.ID.min()
        if len(df)>0:
            dfs.append(df)
        else:
            break
    return pd.concat(dfs,ignore_index=True)

def stock_concepts():
    """所有股票的概念列表"""
    q = query(jy.LC_ConceptList.ConceptCode,jy.LC_ConceptList.ConceptName)
    dict_concept=jy.run_query(q).set_index('ConceptCode')['ConceptName'].to_dict()
    stocks=jy.run_query(query(jy.SecuMain.InnerCode,jy.SecuMain.SecuCode).filter(jy.SecuMain.SecuCategory==1,jy.SecuMain.SecuMarket.in_([83,90]),jy.SecuMain.ListedState.in_([1])))
    s_code=stocks.set_index("InnerCode")['SecuCode']
    df=all_concepts()
    sc=df.groupby('InnerCode').apply(lambda dx:",".join([dict_concept[code] for code in dx.ConceptCode.unique()]))
    df_concept=pd.DataFrame({"concept":sc,'code':s_code})
    df_concept['symbol']=df_concept.code.map(normalize_code,na_action='ignore')
    return df_concept.dropna().set_index('symbol')['concept']
s_concept=stock_concepts()
new_df = pd.DataFrame(s_concept.str.split(',').tolist(), index=s_concept.index).stack()
new_df=new_df.reset_index([0,'symbol'])
new_df.columns=['code','tag']
s_tag=new_df.groupby('tag').apply(lambda dx:list(dx['code'].unique()))

In [ ]:
def concept_hot(date,filter_expr='return3>0.05',sort_col='return3'):
    tags=Tags(date)
    for index, value in s_tag.items():
        if '房价上涨' not in index and '昨日涨停' not in index and '资金龙头' not in index:
            tags.add_tag(index,value)
    return tags.wordcloud(filter_expr=filter_expr,sort_col=sort_col)

In [ ]:
import ipywidgets as widgets
out = widgets.Output()

date=widgets.DatePicker(
    description='选择日期',
    disabled=False
)
def on_value_change(change):
    with out:
        out.clear_output()
        date=change['new']
        out.append_display_data(concept_hot(date,filter_expr='return5>0.05',sort_col='return5'))

date.observe(on_value_change, names='value')
display(date)
display(out)

In [ ]:
text=widgets.Text(
    value='',
    placeholder='概念',
    description='概念名称:',
    disabled=False
)
btn=widgets.Button(
    description='概念股',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='概念股',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
hbox=widgets.HBox([text,btn])

display(hbox)
out1 = widgets.Output()
display(out1)

df=get_all_securities()

def on_button_clicked(b):
    with out1:
        out1.clear_output()
        concept=text.value
        df_select=df[df.index.isin(list(s_tag.loc[concept]))]
        out1.append_display_data(df_select)
btn.on_click(on_button_clicked)

In [9]:
!pip install pyecharts


You should consider upgrading via the 'C:\Users\louis.xu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
